In [1]:
import pandas as pd
from json import loads, dumps

# df = pd.read_csv('/content/ChatGPT Generated - Combined.csv')
df = pd.read_csv('/content/NLPAnnotations - Train_ Complete Cleaned w_ Summaries.csv')


def combine_columns(row):
  """Combine emotions in the 2 columns: New Emotion, Emotion."""
  if pd.isna(row['Emotion']):
    return row['New Emotion']
  elif pd.isna(row['New Emotion']):
    return row['Emotion']
  else:
    return row['Emotion'] + ', ' + row['New Emotion']

df["Combined Emotion"] = df.apply(lambda x: combine_columns(x), axis=1)
df = df.drop(columns=["Emotion", "New Emotion"])

df = df.rename(columns={"ID": "Reddit ID", "Combined Emotion": "Emotion", "selftext_cleaned": "Reddit Post"})
df = df[["Reddit ID", "Title", "Reddit Post", "Summary", "Emotion"]]

pd.set_option('display.max_rows', None)

json_data = df.to_json(orient="records")
parsed = loads(json_data)
pretty_json = dumps(parsed, indent=2)

In [3]:
import json

with open("/content/drive/MyDrive/CovidET-main/data_withposts/train_val_test_anonymized-WITH_POSTS/train_anonymized-WITH_POSTS.json") as f:
    dataset = json.load(f)

ind_to_post_info = {}
size = len(dataset) #get size of original CovidET train dataset

d = {}
#iterate over JSON version of new data and convert to desired JSON format
for i, k in enumerate(parsed):
  d[str(size+i)] = {"Reddit Post": parsed[i]["Reddit Post"], "Annotations": {}}
  if not parsed[i]["Summary"]:
    break
  summaries = parsed[i]["Summary"].split(";")
  d[str(size+i)]["Annotations"]["Annotation 0"] = []
  for eCount, emotion in enumerate(parsed[i]["Emotion"].split(", ")):
    if emotion in {'confusion', 'resilience', 'joy', 'trust', 'disgust'}:
      try:
        d[str(size+i)]["Annotations"]["Annotation 0"].append({"Emotion": emotion.strip(), "Abstractive": summaries[eCount].strip()})
      except:
        pass
json_object = dumps(d, indent=2)
print(json_object)
with open("output.json", "w") as outfile:
    outfile.write(json_object)

{
  "1200": {
    "Reddit Post": "I live in Antivaxxistan. A red red redneck state. And Home Depot is a gathering spot for the antimaskers. Even at the height of the pandemic hardly anyone was wearing masks in Home Depot despite a local city order requiring it. But I had to go there today. I put on my N95 and went in. I was probably the only person in there with a mask. I was looking for some moulding and this old white guy walks by me and says, \"you've been watching too much TV buddy.\" So I went to the back of the store tracked him down and got in his face and pointed at the mask. I said, \"I have cancer.\" He said he was sorry. Fuck him. I don't know if I have cancer today, it's one of those they test me every six months to see if it's back (2.5 years out). But just fuck these people. It only reinforced my militancy on masks.",
    "Annotations": {
      "Annotation 0": [
        {
          "Emotion": "disgust",
          "Abstractive": "I'm disgusted by antimaskers and how even a